## fuckin around

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Rectangle

from collections import defaultdict

# Path Libraries
import pathlib
from pathlib import Path

from typing import Tuple, Dict, List, Union

In [ ]:
#file_name = 'example_data/CHL15-02B-AHe-ZHe-inv.txt'
file_name = 'example_data/single_sample/H5-inv.txt'
encoding = 'latin-1'

In [ ]:
# Read the file contents
with open(file_name, encoding = encoding) as f:
    txt_data_all = f.readlines()

first_word = [r.split()[0] for r in txt_data_all]

first_word

In [ ]:
# Locate key indices for different sections in the file
constraintsIndex = first_word.index('Number')
envelopesIndex = first_word.index('Envelopes')
meanPathIndex = first_word.index('Weighted')
head_length = first_word.index('Fit')

num_constraints = (envelopesIndex-1) - (constraintsIndex+1)

# Extract constraint data
constraints = np.genfromtxt(file_name,
                        delimiter = '\t',
                        skip_header = 2,
                        max_rows = num_constraints,
                        usecols = (1,2,3,4)
                    )

print(constraintsIndex)
print(envelopesIndex)
print(meanPathIndex)
print(head_length)
print(num_constraints)

In [ ]:
constraints

In [ ]:
# Process envelope data
envelope_dict = {}
num_envelopes = (first_word.index('Summaries')-1) - envelopesIndex

print(num_envelopes)


In [ ]:
for i in range(1,num_envelopes+1):
    print(i)

In [ ]:
for i in range(1,num_envelopes+1):
    envelope_data = txt_data_all[envelopesIndex+i].split('\t')
    envelope_data_clean = [entry.strip() for entry in envelope_data if entry.strip()]  # Remove empty and whitespace-only entries
    envelope_label = envelope_data_clean[0]
    
    # Only process non-empty values after the label
    envelope_dict[envelope_label] = [float(value) for value in envelope_data_clean[1:] if value]
    print(envelope_label)

# if num_envelopes == 3 and 'Acc' in txt_data_all[envelopesIndex+i].split('\t')[0]:
#     print('Note that this sample only has acceptable paths.')

In [ ]:
envelope_dict

In [ ]:
grain_headings = first_word.index('Individual')
grain_headings

In [ ]:
sample_grains = txt_data_all[grain_headings+1].split('\t')
sample_grain_stats = txt_data_all[grain_headings+2].split('\t')

sample_grains_cleaned = [
        item.replace('\n', '') if '\n' in item else item for item in sample_grains
    ]

sample_grain_stats_cleaned = [
        item.replace('\n', '') if '\n' in item else item for item in sample_grain_stats
    ]

In [ ]:
sample_grains_cleaned[:10]

In [ ]:
sample_grain_stats_cleaned[:10]

In [ ]:
sample_grain_list = [x for x in sample_grains_cleaned if x]
sample_grain_list

In [ ]:
txt_data = txt_data_all[head_length-1:]

raw_data = [line.split('\t') for line in txt_data[1:]]

data_headers = raw_data[0]

In [ ]:
data_headers[:10]

In [ ]:
# Extract path data
txt_data = txt_data_all[head_length-1:]

raw_data = [line.split('\t') for line in txt_data[1:]]

data_headers = raw_data[0]
path_data = raw_data[1:]
num_paths = (len(path_data))/2

# Identify user-specified constraints in the path data
con_index = [idx for idx, val in enumerate(data_headers) if 'con' in val.lower()]
data_start = path_data[0].index('Time (Ma)')
data_cleaned = []

##
print(data_headers)
print(num_paths)

print(con_index)

In [ ]:
print(data_start)

In [ ]:
# Clean and format path data
for path in path_data:
    cleaned_path = [
        item.replace('\n', '') if '\n' in item else item for item in path
    ]
    formatted_path = [
        item if idx in [0, data_start] else None if idx == 1 and item == '' else float(item)
        for idx, item in enumerate(cleaned_path)
        if len(item) != 0 or idx == 1
    ]
    data_cleaned.append(formatted_path)
    
# Organize path data into dictionary
path_dict = defaultdict(dict)

In [ ]:
data_cleaned[0]

In [ ]:
data_cleaned[1]

In [ ]:
# Loop over each path and populate the dictionary
for path in data_cleaned:
    path_id = path[0]
    path_type = str.lower(path[data_start])

    # Determine the data type (time, temp, or depth) based on the presence of specific keywords
    if 'time (ma)' in path_type:
        data_type_key = 'time'
    elif 'temp' in path_type:
        data_type_key = 'temp'
    elif 'depth' in path_type:
        data_type_key = 'depth'
    else:
        # Skip entries without recognized types
        continue

    # Store main data for time, temp, or depth
    path_dict[path_id][data_type_key] = path[data_start + 1:]

    # Store 'stage' for time paths, 'comp_GOF' for temp and depth
    if data_type_key == 'time':
        path_dict[path_id]['stage'] = path[1]
    else:
        path_dict[path_id]['comp_GOF'] = path[1]

    # Collect user-specified constraint values and store them under the constraint-specific key
    con_only = [val for idx, val in enumerate(path) if idx in con_index]
    path_dict[path_id][f'{data_type_key}_con'] = con_only

In [ ]:
test_data = data_cleaned[0]

In [ ]:
path_id = test_data[0]
path_type = str.lower(test_data[data_start])

print(path_id)
print(path_type)

In [ ]:
print(sample_grains_cleaned[2])
print(sample_grains_cleaned[data_start-1])

In [ ]:
print(sample_grain_stats_cleaned[2])
print(sample_grain_stats_cleaned[data_start-1])

In [ ]:
sample_grains_cleaned = [s.replace(' ', '_') if s else s for s in sample_grains_cleaned]

In [ ]:
sample_grain_stats_cleaned = [s.replace(' ', '_') if s else s for s in sample_grain_stats_cleaned]

In [ ]:
for i in range(2,data_start):
    print(sample_grains_cleaned[i],",",sample_grain_stats_cleaned[i], data_headers[i])

In [ ]:
sample_grains_cleaned

In [ ]:
sample_grain_list = [x for x in sample_grains_cleaned if x]
sample_grain_list

In [ ]:
seen = set()
sample_grain_list = [x for x in sample_grains_cleaned if x and not (x in seen or seen.add(x))]

In [ ]:
sample_grain_list

In [ ]:
# Determine the data type (time, temp, or depth) based on the presence of specific keywords
if 'time (ma)' in path_type:
    data_type_key = 'time'
elif 'temp' in path_type:
    data_type_key = 'temp'
elif 'depth' in path_type:
    data_type_key = 'depth'
else:
    # Skip entries without recognized types
    pass

# Store main data for time, temp, or depth
path_dict[path_id][data_type_key] = test_data[data_start + 1:]

# Store 'stage' for time paths, 'comp_GOF' for temp and depth
if data_type_key == 'time':
    path_dict[path_id]['stage'] = test_data[1]
    for i in range(2,data_start):
        grain_id = sample_grains_cleaned[i]
        grain_stat1 = sample_grain_stats_cleaned[i]
        
        if grain_id not in path_dict[path_id]:
            path_dict[path_id][grain_id] = {}
            
        if grain_stat1 in ('corr._age_(Ma)','age_(Ma)'):
            path_dict[path_id][grain_id]['modeled_age'] = float(path[i]) if path[i] else None
        else:
            path_dict[path_id][grain_id][grain_stat1] = float(path[i]) if path[i] else None
            
        #path_dict[path_id][grain_id] = {'corr_modeled_age': test_data[i]}
else:
    path_dict[path_id]['comp_GOF'] = test_data[1]
    for i in range(2,data_start):
        grain_id = sample_grains_cleaned[i]
        grain_stat2 = data_headers[i]
        
        if grain_id not in path_dict[path_id]:
            path_dict[path_id][grain_id] = {}
        
        if grain_stat2 == 'GOF':
            #path_dict[path_id][grain_id] = {'modeled_age_GOF': test_data[i]}
            path_dict[path_id][grain_id]['modeled_age_GOF'] = float(path[i]) if path[i] else None
        else:
            path_dict[path_id][grain_id][grain_stat2] = float(path[i]) if path[i] else None
        
# Collect user-specified constraint values and store them under the constraint-specific key
con_only = [val for idx, val in enumerate(path) if idx in con_index]
path_dict[path_id][f'{data_type_key}_con'] = con_only

In [ ]:
path_dict

In [ ]:
ax.set_xlabel('Time (Ma)',
                    labelpad = 8.0,
                    fontname= "Arial",
                    fontsize=12,
                    weight=500,
                    color=GREY40
                    )

In [ ]:
title = 'Sample {} - {}'.format(sample, plot_type)
if plotAgeHistogram:
    fig.suptitle(title,
              fontsize = 15,
              fontname = "Arial",
              color = GREY10
              )
else:
    plt.title(title,
              fontsize = 15,
              fontname = "Arial",
              color = GREY10,
              pad = 15
             )

## 0. Import modules
Run this cell to import the necessary Jupyter modules every time you open HeFTyPy.

In [1]:
# HeFTy Functions
import HeFTyFuncsClasses as hFunc

import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## <font color = green> *Single Sample Model Workflow* </font>
### 1. Load HeFTy Model Data

#### Testing Helium Model

In [ ]:
file_name = 'example_data/CHL15-02B-AHe-ZHe-inv.txt'

# Create a model holding all of the data
model_data = hFunc.SingleSampleModel(file_name)

In [ ]:
test_data = model_data.get_path_data()

In [ ]:
test_data['Best']

#### Testing Fission Track Model

In [ ]:
file_name_AFT = 'example_data/AL01_AFT-inv_tZ.txt'

# Create a model holding all of the data
model_data_AFT = hFunc.SingleSampleModel(file_name_AFT)

In [ ]:
test_data_AFT = model_data_AFT.get_path_data()

In [ ]:
test_data_AFT['Best']

### Plot Distribution of Modeled Ages

#### Testing Helium Model

In [ ]:
bins = 20
share_x = False

whatToPlot = 'histogram' # 'histogram', 'kde', 'both'

# Save Figure 
saveFig = False
saveFolder = 'Plots_testing'
savefigFileName = 'CHL15_modeled_ages_histogram'

##
model_data.plot_modeled_age_histograms(bins, share_x, whatToPlot, saveFig, saveFolder, savefigFileName)

#### Testing Fission Track Model

In [ ]:
bins = 20
share_x = False

whatToPlot = 'both' # 'histogram', 'kde', 'both'

# Save Figure 
saveFig = False
saveFolder = 'Plots_testing'
savefigFileName = 'CHL15_modeled_ages_histogram'

##
model_data_AFT.plot_modeled_age_histograms(bins, share_x, whatToPlot, saveFig, saveFolder, savefigFileName)

### Measured vs. Modeled Ages

#### Testing Helium Model
Run the below cell to get a list of samples included in the model. 

In [ ]:
model_data.sample_grain_list

In [ ]:
# Specify measured ages for samples in model 
measured_sample_ages = [15.1, 25.8, 12.8, 15.0, 204.6, 245.2, 243.5] # Ages should correspond to above sample list, in order

# Specify axis bounds
ahe_x_bounds = (10,30)
zhe_x_bounds = (200,250)

ahe_y_bounds = (0,35)
zhe_y_bounds = (100,300)

aft_x_bounds = None
zft_x_bounds = None

aft_y_bounds = None
zft_y_bounds = None

show_1v1_line = 'both' # 'line', 'point', 'both' or None

# Save Figure 
saveFig = False
saveFolder = 'Plots_testing'
savefigFileName = 'CHL15_measured_vs_modeled ages'

In [ ]:
model_data.plot_measured_vs_modeled(measured_sample_ages, 
                                    ahe_x_bounds, zhe_x_bounds, aft_x_bounds, zft_x_bounds,
                                    ahe_y_bounds, zhe_y_bounds, aft_y_bounds, zft_y_bounds,
                                    show_1v1_line,
                                    saveFig, saveFolder, savefigFileName)

#### Testing Fission Track Model
Run the below cell to get a list of samples included in the model. 

In [ ]:
model_data_AFT.sample_grain_list

In [ ]:
# Specify measured ages for samples in model 
measured_sample_ages = [35,40] # Ages should correspond to above sample list, in order

# Specify axis bounds
ahe_x_bounds = None
zhe_x_bounds = None

ahe_y_bounds = None
zhe_y_bounds = None

aft_x_bounds = (25,50)
zft_x_bounds = None

aft_y_bounds = (20,50)
zft_y_bounds = None

show_1v1_line = 'both' # 'line', 'point', 'both' or None

# Save Figure 
saveFig = False
saveFolder = 'Plots_testing'
savefigFileName = 'AL01_measured_vs_modeled ages'

In [ ]:
model_data_AFT.plot_measured_vs_modeled(measured_sample_ages, 
                                    ahe_x_bounds, zhe_x_bounds, aft_x_bounds, zft_x_bounds,
                                    ahe_y_bounds, zhe_y_bounds, aft_y_bounds, zft_y_bounds,
                                    show_1v1_line,
                                    saveFig, saveFolder, savefigFileName)

### 2. Visualize Single Sample Data
This function plots HeFTy path data for single sample models. There are options to plot the data as lines, points, or envelopes. 

#### Testing Helium Model

In [ ]:
## Set sample and plot details -----------------------------
sample = 'CHL15' 

plot_type = 'line' # 'line', 'points', or 'envelopes'
y_variable = 'temp' # 'temp' or 'depth'

which_envelopes = 'both' #'both', 'good', or 'acc'

y_lim = (300,0) # (higher bound, lower bound)
x_lim = (290,0) # (higher bound, lower bound)

## Histogram (optional) -----------------------------
plotAgeHistogram = True # True or False
bin_width = 5
histogramColorPalette = 'Dark2'
stat = 'count' # 'count', 'frequency', 'probability', 'percent', or 'density'

## Customize plot style -----------------------------
# default HeFty style is black best path, fuchsia good paths, and green acceptable paths 
defaultHeftyStyle = True # True or False 

# the following colors will only apply if defaultHeftyStyle (above) is marked False
bestPathColor = 'midnightblue'
goodPathColor = 'cornflowerblue'
accPathColor = 'lightsteelblue'

# Constraint Box Settings
showConstraintBoxes = True # True or False
constraintBoxColor = 'red'
constraintLineColor = 'black'
constraintMarkerStyle = 's'

## Save Figure -----------------------------
saveFig = False
saveFolder = 'Plots_testing'
savefigFileName = 'CHL15_Path_data_test_lines_w_histogram'

In [ ]:
model_data.plotSingleSamplePathData(sample, plot_type, 
                            y_variable, which_envelopes, y_lim, x_lim,
                            plotAgeHistogram, bin_width, histogramColorPalette, stat,
                            defaultHeftyStyle, bestPathColor , goodPathColor , accPathColor ,
                            showConstraintBoxes, constraintBoxColor , constraintLineColor , constraintMarkerStyle ,
                            saveFig, saveFolder, savefigFileName)

#### Testing Fission Track Model

In [ ]:
## Set sample and plot details -----------------------------
sample = 'AL01' 

plot_type = 'points' # 'line', 'points', or 'envelopes'
y_variable = 'depth' # 'temp' or 'depth'

which_envelopes = 'both' #'both', 'good', or 'acc'

y_lim = (30,0) # (higher bound, lower bound)
x_lim = (310,0) # (higher bound, lower bound)

## Histogram (optional) -----------------------------
plotAgeHistogram = True # True or False
bin_width = 5
histogramColorPalette = 'Dark2'
stat = 'count' # 'count', 'frequency', 'probability', 'percent', or 'density'

## Customize plot style -----------------------------
# default HeFty style is black best path, fuchsia good paths, and green acceptable paths 
defaultHeftyStyle = True # True or False 

# the following colors will only apply if defaultHeftyStyle (above) is marked False
bestPathColor = 'midnightblue'
goodPathColor = 'cornflowerblue'
accPathColor = 'lightsteelblue'

# Constraint Box Settings
showConstraintBoxes = True # True or False
constraintBoxColor = 'red'
constraintLineColor = 'black'
constraintMarkerStyle = 's'

## Save Figure -----------------------------
saveFig = False
saveFolder = 'Plots_testing'
savefigFileName = 'AL01_Path_data_test_lines_w_histogram'

In [ ]:
model_data_AFT.plotSingleSamplePathData(sample, plot_type, 
                            y_variable, which_envelopes, y_lim, x_lim,
                            plotAgeHistogram, bin_width, histogramColorPalette, stat,
                            defaultHeftyStyle, bestPathColor , goodPathColor , accPathColor ,
                            showConstraintBoxes, constraintBoxColor , constraintLineColor , constraintMarkerStyle ,
                            saveFig, saveFolder, savefigFileName)

### 3. Identify Single-Sample Path Families

#### Testing Helium Model

In [ ]:
## Set sample and plot details -----------------------------
sample = 'CHL15'

plot_type = 'line' #'line' or 'points'
y_variable = 'temp' #'temp' or'depth

y_lim = (300,0) # (higher bound, lower bound)
x_lim = (290,0) # (higher bound, lower bound)


## Customize plot style -----------------------------
good_match_color = 'goldenrod'
acc_match_color = 'gold'

showConstraintBoxes = True
constraintBoxColor = 'red'
constraintLineColor = 'black'
constraintMarkerStyle = 's'

## Specify bounds for path families -----------------------------
# Set bounds for each constraint box to identify path families. Default is None.
# Constraints set in the style of ('<= value', '>= value') i.e. between (higher number, lower number)
# Function can accomodate up to 15 constraints, add additional constraints in format of c1_x/c1_y ... c15_x/x15_y
# If you add more constraints, make sure to add them to the end of the function call as well. 
c1_x =  (15,0)
c1_y = None

c2_x = None
c2_y = None

c3_x = None 
c3_y = None

c4_x = None 
c4_y = None

## Save Figure -----------------------------
saveFig = False
saveFolder = 'Plots_testing'
savefigFileName = 'CHL15_matched_paths_<15Ma'

In [ ]:
key_matched_paths = model_data.identifyPathFamilies(sample, plot_type,
                              y_variable, y_lim, x_lim ,
                              good_match_color, acc_match_color, 
                              showConstraintBoxes, 
                              constraintBoxColor, constraintLineColor, constraintMarkerStyle,
                              saveFig, saveFolder, savefigFileName,
                              c1_x, c1_y,c2_x, c2_y,c3_x, c3_y, c4_x, c4_y)

In [ ]:
key_matched_paths

#### Testing Fission Track Model

In [ ]:
## Set sample and plot details -----------------------------
sample = 'AL01'

plot_type = 'points' #'line' or 'points'
y_variable = 'depth' #'temp' or'depth

y_lim = (30,0) # (higher bound, lower bound)
x_lim = (310,0) # (higher bound, lower bound)


## Customize plot style -----------------------------
good_match_color = 'goldenrod'
acc_match_color = 'gold'

showConstraintBoxes = True
constraintBoxColor = 'red'
constraintLineColor = 'black'
constraintMarkerStyle = 's'

## Specify bounds for path families -----------------------------
# Set bounds for each constraint box to identify path families. Default is None.
# Constraints set in the style of ('<= value', '>= value') i.e. between (higher number, lower number)
# Function can accomodate up to 15 constraints, add additional constraints in format of c1_x/c1_y ... c15_x/x15_y
# If you add more constraints, make sure to add them to the end of the function call as well. 
c1_x =  (20,0)
c1_y = None

c2_x = None
c2_y = None

c3_x = None 
c3_y = None

c4_x = None 
c4_y = None

## Save Figure -----------------------------
saveFig = False
saveFolder = 'Plots_testing'
savefigFileName = 'AL01_matched_paths_<15Ma'

In [ ]:
key_matched_paths_AFT = model_data_AFT.identifyPathFamilies(sample, plot_type,
                              y_variable, y_lim, x_lim ,
                              good_match_color, acc_match_color, 
                              showConstraintBoxes, 
                              constraintBoxColor, constraintLineColor, constraintMarkerStyle,
                              saveFig, saveFolder, savefigFileName,
                              c1_x, c1_y,c2_x, c2_y,c3_x, c3_y, c4_x, c4_y)

In [ ]:
key_matched_paths_AFT

---

## <font color = purple> *Multi Sample Model Workflow* </font>
### Figuring out Multi-Sample Class Model
#### Load in the multi-sample model

In [2]:
folder_path = 'example_data/multi_sample'

# Create a model holding all of the data
multisample_model = hFunc.MultiSampleModel(folder_path)

#### Check the samples and data within the larger model

In [3]:
multisample_model.list_samples_and_types()

[('GAN', ['temp']),
 ('AN03', ['temp', 'depth']),
 ('AN04', ['temp']),
 ('AN05', ['temp'])]

See that each sample is linked to a SingleSampleModel object. 

In [4]:
multisample_model.samples

{'GAN': {'temp': <HeFTyFuncsClasses.SingleSampleModel at 0x16e83c3d0>},
 'AN03': {'temp': <HeFTyFuncsClasses.SingleSampleModel at 0x1039781d0>,
  'depth': <HeFTyFuncsClasses.SingleSampleModel at 0x176fa4190>},
 'AN04': {'temp': <HeFTyFuncsClasses.SingleSampleModel at 0x1039b5e50>},
 'AN05': {'temp': <HeFTyFuncsClasses.SingleSampleModel at 0x177cdc110>}}

#### Identify the 'Master Sample' that has info on constraints, and models for depth and temperature

In [5]:
multisample_model.master_sample

{'sample_name': 'AN03',
 'models': {'temp': <HeFTyFuncsClasses.SingleSampleModel at 0x1039781d0>,
  'depth': <HeFTyFuncsClasses.SingleSampleModel at 0x176fa4190>}}

#### Check the best fit paths

In [8]:
multisample_model.best_paths

{'GAN': {'temp': {'time': [275.765962567119,
    274.96486872049,
    274.803414297918,
    274.695778016203,
    274.641959875345,
    274.588141734488,
    274.480505452773,
    274.372869171058,
    274.265232889343,
    274.157596607628,
    274.049960325913,
    273.834687762483,
    273.626845838437,
    273.251845838437,
    273.023686733023,
    272.795527627609,
    272.567368522195,
    272.339209416781,
    272.111050311367,
    271.882891205953,
    271.654732100539,
    271.426572995125,
    271.198413889712,
    270.970254784298,
    270.742095678884,
    270.51393657347,
    270.057618362642,
    269.601300151814,
    268.968117675,
    268.364958569586,
    267.908640358758,
    266.996003937103,
    266.083367515447,
    265.170731093791,
    264.029935566722,
    262.660980934238,
    261.292026301755,
    259.694912563858,
    258.247045560008,
    256.878090927524,
    255.280977189627,
    253.945704516939,
    253.036227200697,
    251.667272568214,
    249.841999

In [13]:
multisample_model.get_sample_best_path('AN03','depth')

{'time': [275.7659626,
  275.150542,
  275.072505,
  273.6268458,
  268.9681177,
  259.3065229,
  254.0957045,
  226.2515525,
  100.6897334,
  97.46640825,
  96.21698689,
  61.24766894,
  13.97131073,
  12.94628487,
  11.52808817,
  11.28262697,
  0.0],
 'stage': '',
 'He_Apatite': {'modeled_age': 15.25022125, 'modeled_age_GOF': 0.38088736},
 'time_con': [275.7659626, 268.9681177, 100.6897334, 13.97131073, 0.0],
 'depth': [29.67390802,
  21.19355594,
  15.78314755,
  10.41076144,
  9.604166165,
  8.172119381,
  7.549111003,
  6.261754984,
  5.88947616,
  5.403253941,
  4.382773266,
  3.1308573,
  2.90919385,
  2.856558232,
  2.493547954,
  2.438226134,
  0.0],
 'comp_GOF': 0.77262333,
 'depth_con': [29.67390802, 9.604166165, 5.88947616, 2.90919385, 0.0]}

#### Pull one of the samples into its own single sample model 

In [ ]:
model_data_test = multisample_model.samples['GAN']['temp']

In [ ]:
test = multisample_model.get_sample('GAN')

### 2. Visualize Multi-Sample Model Data
### *Plot Distribution of Modeled Ages for each Sample within the Multi-Sample Model*

In [ ]:
# Specify sample and variable from the multi-sample model
sample = 'AN03'
variable = 'temp'

# Specify histogram parameters
bins = 20
share_x = False

whatToPlot = 'both' # 'histogram', 'kde', 'both'

# Save Figure 
saveFig = False
saveFolder = 'Plots_testing'
savefigFileName = 'CHL15_modeled_ages_histogram'

In [ ]:
multisample_model.plotMultiSampleModeledAgeHistogram(sample,variable,bins, share_x, whatToPlot, saveFig, saveFolder, savefigFileName)

### *Plot Measured vs. Modeled Ages for each Sample within the Multi-Sample Model*

In [ ]:
# Specify sample and variable from the multi-sample model
sample = 'AN03'
variable = 'temp'

# Specify measured ages for samples in model - Ages should correspond to above sample list, in order
measured_sample_ages = [15.1] 

pathsToPlot = 'all' # 'good', 'acc', or 'all' – best path will be shown for all options

colorPalette = 'Set1' 

# Specify plot parameters
ahe_x_bounds = (10,20)
ahe_y_bounds = (0,30)

zhe_x_bounds = None
zhe_y_bounds = None

aft_x_bounds = None
aft_y_bounds = None

zft_x_bounds = None
zft_y_bounds = None

show_1v1_line = 'both' # 'line', 'point', 'both' or None

# Save Figure 
saveFig = False
saveFolder = 'Plots_testing'
savefigFileName = 'CHL15_measured_vs_modeled ages'

In [ ]:
multisample_model.plotMultiSampleModeled_v_Measured(sample, variable, measured_sample_ages, 
                                                    ahe_x_bounds, zhe_x_bounds, aft_x_bounds, zft_x_bounds,
                                                    ahe_y_bounds, zhe_y_bounds, aft_y_bounds, zft_y_bounds,
                                                    show_1v1_line,
                                                    saveFig, saveFolder, savefigFileName)

In [ ]:
model_data_test.sample_grain_list

In [ ]:
constraints = model_data_test.get_constraints()
print(constraints)

#### Plots for the subset(?) model 

In [ ]:
# Specify histogram parameters
bins = 20
share_x = False

whatToPlot = 'histogram' # 'histogram', 'kde', 'both'

# Save Figure 
saveFig = True
saveFolder = 'Plots_testing'
savefigFileName = 'GAN_temp_modeled_ages_histogram'

In [ ]:
model_data_test.plot_modeled_age_histograms(bins, share_x, whatToPlot, saveFig, saveFolder, savefigFileName)

In [ ]:
## Set sample and plot details -----------------------------
sample = 'GAN' 

plot_type = 'line' # 'line', 'points', or 'envelopes'
y_variable = 'temp' # 'temp' or 'depth'

which_envelopes = 'both' #'both', 'good', or 'acc'

y_lim = (300,0) # (higher bound, lower bound)
x_lim = (285,0) # (higher bound, lower bound)

## Histogram (optional) ----------------------------- (only for master sample in time-depth space)
plotAgeHistogram = False # True or False
bin_width = 5
histogramColorPalette = 'Dark2'
stat = 'count' # 'count', 'frequency', 'probability', 'percent', or 'density'

## Customize plot style -----------------------------
# default HeFty style is black best path, fuchsia good paths, and green acceptable paths 
defaultHeftyStyle = True # True or False 

# the following colors will only apply if defaultHeftyStyle (above) is marked False
bestPathColor = 'midnightblue'
goodPathColor = 'cornflowerblue'
accPathColor = 'lightsteelblue'

# Constraint Box Settings
showConstraintBoxes = True # True or False
constraintBoxColor = 'red'
constraintLineColor = 'black'
constraintMarkerStyle = 's'

## Save Figure -----------------------------
saveFig = False
saveFolder = 'Plots_testing'
savefigFileName = 'GAN_Path_data_test_lines_w_histogram'

In [ ]:
model_data_test.plotSingleSamplePathData(sample, plot_type, 
                            y_variable, which_envelopes, y_lim, x_lim,
                            plotAgeHistogram, bin_width, histogramColorPalette, stat,
                            defaultHeftyStyle, bestPathColor , goodPathColor , accPathColor ,
                            showConstraintBoxes, constraintBoxColor , constraintLineColor , constraintMarkerStyle ,
                            saveFig, saveFolder, savefigFileName)